### 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
임베딩 벡터를 얻기 위해서 파이토치의 nn.Embedding()을 사용하기도 하지만, 때로는 이미 훈련되어져 있는 워드 임베딩을 불러서 이를 임베딩 벡터로 사용하기도 합니다. 훈련 데이터가 부족한 상황이라면 모델에 파이토치의 nn.Embedding()을 사용하는 것보다 다른 텍스트 데이터로 사전 훈련되어 있는 임베딩 벡터를 불러오는 것이 나은 선택일 수 있습니다.

훈련 데이터가 적다면 파이토치의 nn.Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않습니다. 이 경우, 해당 문제에 특화된 것은 아니지만 보다 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있습니다.

In [1]:
!pip install gensim

### 1. 사전 훈련된 임베딩을 사용하지 않는 경우

In [2]:
import numpy as np
from collections import Counter
import gensim

문장의 긍, 부정을 판단하는 감성 분류 모델을 만들어봅시다. 문장과 레이블 데이터를 만들었습니다. 긍정인 문장은 레이블 1, 부정인 문장은 레이블이 0입니다.

In [3]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

각 샘플에 대해서 단어 토큰화를 수행합니다.

In [4]:
tokenized_sentences = [sent.split( ) for sent in sentences]
print('단어 토큰화된 결과: ', tokenized_sentences)

단어 토큰화된 결과:  [['nice', 'great', 'best', 'amazing'], ['stop', 'lies'], ['pitiful', 'nerd'], ['excellent', 'work'], ['supreme', 'quality'], ['bad'], ['highly', 'respectable']]


토큰화 된 결과를 바탕으로 단어 집합을 만들어봅시다. 우선 Counter() 모듈을 이용하여 각 단어의 등장 빈도수를 기록합니다.

In [5]:
word_list = []
for sent in tokenized_sentences:
  for word in sent:
    word_list.append(word)

word_counts = Counter(word_list)
print('총 단어의수 :', len(word_counts))

총 단어의수 : 15


현재 존재하는 총 단어의 수는 15개입니다. 이 단어들을 등장 빈도가 높은 순서부터 정렬합니다.

In [6]:
# 등장 빈도순 정렬
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab)

['nice', 'great', 'best', 'amazing', 'stop', 'lies', 'pitiful', 'nerd', 'excellent', 'work', 'supreme', 'quality', 'bad', 'highly', 'respectable']


nice가 등장 빈도수로 가장 높은 단어이고, 그 다음은 great, 그 다음은 best로 등장 빈도가 높은 순서대로 단어가 정렬된 상태입니다. 이제 이로부터 단어 집합을 완성해봅시다. 0번은 패딩 토큰을 위한 용도로 사용하고, 1번은 단어 집합에 없는 단어가 등장하는 OOV(Out-Of-Vocabulary) 문제가 발생하면 사용하는 용도로 각각 할당합니다.

In [7]:
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

for index, word in enumerate(vocab):
  word_to_index[word] = index +2

vocab_size = len(word_to_index)
print('패딩 토큰, UNK 토큰을 고려한단어 집합의 크기: ', vocab_size)

패딩 토큰, UNK 토큰을 고려한단어 집합의 크기:  17


In [8]:
print(word_to_index)

{'<PAD>': 0, '<UNK>': 1, 'nice': 2, 'great': 3, 'best': 4, 'amazing': 5, 'stop': 6, 'lies': 7, 'pitiful': 8, 'nerd': 9, 'excellent': 10, 'work': 11, 'supreme': 12, 'quality': 13, 'bad': 14, 'highly': 15, 'respectable': 16}


단어 집합을 이용하여 정수 인코딩을 진행합니다. 단어 집합에 없는 단어가 등장할 경우에는 정수 1이 할당되지만 이번 실습에서는 학습 데이터에 단어 집합에 없는 단어가 존재하지 않으므로 해당되지 않습니다.

In [10]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
        index_sequences.append(word_to_index[word])
      except:
        index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

X_encoded= texts_to_sequences(tokenized_sentences, word_to_index)
print(X_encoded)

[[2, 3, 4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14], [15, 16]]


현재 데이터의 최대 길이를 측정하고, 해당 길이로 패딩을 진행합니다.

In [11]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이: ', max_len)

최대 길이:  4


In [12]:
def pad_sequences(sentences,max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index,:len(sentence)] = np.array(sentence)[:max_len]
  return features

X_train = pad_sequences(X_encoded, max_len)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 2  3  4  5]
 [ 6  7  0  0]
 [ 8  9  0  0]
 [10 11  0  0]
 [12 13  0  0]
 [14  0  0  0]
 [15 16  0  0]]


모든 데이터의 길이가 4로 변환된 것을 확인하였습니다. 이제 nn.Embedding()를 이용하여 모델을 설계합니다.

In [13]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

In [17]:
class SimpleModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(SimpleModel, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.flatten = nn.Flatten() # 평평하게 1차원으로 변경
    self.fc = nn.Linear(embedding_dim*max_len, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
     # embedded.shape == (배치 크기, 문장의 길이, 임베딩 벡터의 차원)
     embedded = self.embedding(x)

     # flattend.shape == (배치 크기, 문장의 길이 × 임베딩 벡터의 차원)
     flattened = self.flatten(embedded)

     # output.shape == (배치 크기, 1)
     output = self.fc(flattened)
     return self.sigmoid(output)

모델 객체를 선언합니다. 임베딩 벡터의 크기는 100으로 정했습니다.

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embedding_dim = 100
simple_model= SimpleModel(vocab_size, embedding_dim).to(device)

출력층에 로지스틱 회귀를 이용한 이진 분류 문제를 푸는 모델이므로 손실 함수로는 바이너리 크로스엔트로피 함수에 해당하는 nn.BCELoss()를 사용합니다.

In [19]:
criterion = nn.BCELoss()
optimizer = Adam(simple_model.parameters())

데이터를 배치 크기 2로 설정한 데이터로더로 변환합니다.

In [20]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long),
                              torch.tensor(y_train, dtype=torch.float32))

train_dataloader = DataLoader(train_dataset, batch_size = 2)

데이터가 7개였으므로 배치 크기 2로 묶으면 총 묶음은 4개(2개, 2개, 2개, 1개)가 됩니다.

In [21]:
print(len(train_dataloader))

4


총 10번 학습합니다.

In [22]:
for epoch in range(10):
  for inputs, targets in train_dataloader:
    # inputs.shape == (배치크기, 문장 길이)
    # targets.shape == (배치크기)
    inputs, targets = inputs.to(device), targets.to(device)

    optimizer.zero_grad()

    #outputs.shape == (배치 크기)
    outputs = simple_model(inputs).view(-1)

    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 1.3473931550979614
Epoch 2, Loss: 1.0007615089416504
Epoch 3, Loss: 0.71290522813797
Epoch 4, Loss: 0.5088858604431152
Epoch 5, Loss: 0.3772549033164978
Epoch 6, Loss: 0.29641881585121155
Epoch 7, Loss: 0.24746374785900116
Epoch 8, Loss: 0.21727779507637024
Epoch 9, Loss: 0.19745640456676483
Epoch 10, Loss: 0.18272405862808228


###2. 사전 훈련된 임베딩을 사용하는 경우
구글에서 사전 학습시킨 Word2Vec 모델을 사용하여 문제를 풀어봅시다. 우선 구글에서 사전 학습시킨 Word2Vec 모델을 다운로드 합니다.

In [23]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j


Downloading...
From (original): https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j
From (redirected): https://drive.google.com/uc?id=1Av37IVBQAAntSe1X3MOAl5gvowQzd2_j&confirm=t&uuid=61ff900d-a99c-4bd9-9126-c07e5ae2cbef
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:30<00:00, 53.6MB/s]


In [24]:
# 구글에 사전 훈련되 Word2vec 모델을 로드 합니다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

위 모델은 각 벡터가 300차원으로 구성되어져 있습니다. 풀고자 하는 문제의 단어 집합 크기의 행과 300개의 열을 가지는 행렬 생성합니다. 이 행렬의 값은 전부 0으로 채웁니다. 이 행렬에 사전 훈련된 임베딩 값을 넣어줄 것입니다.

In [26]:
embedding_matrix = np.zeros((vocab_size, 300))
print('임베딩 행렬의 크기: ', embedding_matrix.shape)

임베딩 행렬의 크기:  (17, 300)


word2vec_model에서 특정 단어를 입력하면 해당 단어의 임베딩 벡터를 리턴받을텐데, 만약 word2vec_model에 특정 단어의 임베딩 벡터가 없다면 None을 리턴하도록 하는 함수 get_vector()를 구현합니다.

In [27]:
def get_vector(word):
  if word in word2vec_model:
    return word2vec_model[word]

  else:
    return None

단어 집합으로부터 단어를 1개씩 호출하여 word2vec_model에 해당 단어의 임베딩 벡터값이 존재하는지 확인합니다. 만약 None이 아니라면 존재한다는 의미이므로 임베딩 행렬에 해당 단어의 인덱스 위치의 행에 임베딩 벡터의 값을 저장합니다.

In [28]:
# <PAD>를 위한 0번과 <UNK>를 위한 1번은 실제 단어가 아니므로 맵핑에서 제외
for word, i in word_to_index.items():
  if i > 2:
    temp = get_vector(word)
    if temp is not None:
      embedding_matrix[i] = temp

현재 풀고자하는 문제의 17개의 단어와 맵핑되는 임베딩 행렬이 완성됩니다. 0번 단어는 패딩을 위한 용도이므로 사전 훈련된 임베딩 벡터값이 불필요합니다. 이에 따라 초기값인 0벡터로 초기화가 되어져 있습니다. embedding_matrix의 0번 위치의 벡터를 출력해봅시다.

In [29]:
# <PAD>나 <UNK>의 경우는 사전 훈련된 임베딩이 들어가지 않아서 0벡터임
print(embedding_matrix[0])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


0이 300개 채워진 벡터임을 확인하였습니다. 이제 다른 단어들도 제대로 맵핑이 됐는지 확인해볼까요? 기존의 단어 집합에서 단어 'great'가 정수로 몇 번인지 확인합니다.

In [30]:
word_to_index['great']

3

3번 임을 확인했습니다. 이에 따라서 사전 훈련된 word2vec_model에서의 'great' 벡터와 현재 사전 훈련된 임베딩 벡터가 맵핑된 embedding_matrix의 3번 벡터가 동일한지 확인합니다.

In [31]:
# word2vec_model에서 'great'의 임베딩 벡터
# embedding_matrix[3]이 일치하는지 체크
np.all(word2vec_model['great'] == embedding_matrix[3])


True

동일한 것을 확인하였습니다. 이는 현재 3번 위치에 단어 'great' 벡터가 정상적으로 할당되었음을 의미합니다. 이제 사전 훈련된 임베딩을 이용한 모델을 구현합니다.

In [58]:
class PretrainedEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(PretrainedEmbeddingModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(embedding_dim * max_len, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        flattened = self.flatten(embedded)
        output = self.fc(flattened)
        return self.sigmoid(output)


모델 객체를 선언합니다. 이때 임베딩 벡터의 크기는 embedding_matrix에서 이미 정해진 임베딩 벡터의 차원인 300으로 해야만 합니다.

In [59]:
pretraiend_embedding_model = PretrainedEmbeddingModel(vocab_size, 300).to(device)

출력층에 로지스틱 회귀를 이용한 이진 분류 문제를 푸는 모델이므로 손실 함수로는 바이너리 크로스엔트로피 함수에 해당하는 nn.BCELoss()를 사용합니다.

In [60]:
criterion = nn.BCELoss()
optimizer = Adam(pretraiend_embedding_model.parameters())

데이터를 배치크기 2로 설정한 데이터로더 변환합니다.

In [61]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.long),
                              torch.tensor(y_train, dtype=torch.float32))
train_dataloader = DataLoader(train_dataset, batch_size=2)

In [62]:
print(len(train_dataloader))

4


총 10번에 학습 수행

In [63]:
for epoch in range(10):
    for inputs, targets in train_dataloader:
        # inputs.shape == (배치 크기, 문장 길이)
        # targets.shape == (배치 크기)
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        # outputs.shape == (배치 크기)
        outputs = pretraiend_embedding_model(inputs).view(-1)

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.7093459963798523
Epoch 2, Loss: 0.64645916223526
Epoch 3, Loss: 0.5838637948036194
Epoch 4, Loss: 0.5248922109603882
Epoch 5, Loss: 0.47041550278663635
Epoch 6, Loss: 0.42063507437705994
Epoch 7, Loss: 0.37548696994781494
Epoch 8, Loss: 0.3347833454608917
Epoch 9, Loss: 0.298271119594574
Epoch 10, Loss: 0.26566213369369507


사전 훈련된 임베딩을 이용하여 기존 모델 대비 더 높은 성능을 얻는 예시는 '사전 훈련된 임베딩을 이용한 성능 상승 시키기(https://wikidocs.net/217099)' 실습을 참고하시기 바랍니다.